In [5]:
import requests
import time
from bs4 import BeautifulSoup
import forecastio
import numpy
import pandas as pd

#### 네이버 주식 가져오기
- api 사용: json을 한다
- 네이버주식 페이지에서 주식 데이터를 가져와 데이터 프레임으로 만들기
- http://m.stock.naver.com

In [8]:
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?\
menu=market_sum&sosok=0&pageSize=" + str(pageSize) + "&page=" + str(page)
#     http://m.stock.naver.com/sise/siseList.nhn?menu=market_sum&sosok=0

def get_data(url):
    response = requests.get(url)
    json_info = response.json()
    companys = json_info['result']['itemList']
    df = pd.DataFrame(columns = ['종목','시세','전일비','등락율','시가총액','거래량'])
    for company in companys:
        df.loc[len(df)] = {
            '종목': company['nm'],
            '시세': company['nv'],
            '전일비': company['cv'],
            '등락율': company['cr'],
            '시가총액': company['mks'],
            '거래량': company['aq'],
        }
    return df

In [10]:
url = make_url(1406,1)
df = get_data(url)
print(len(df))
df.tail()

1406


,종목,시세,전일비,등락율,시가총액,거래량
1401,KODEX 골드선물인버스(H),10140,60,0.60,51,1046
1402,대신 인버스 니켈선물 ETN(H),7180,40,0.56,50,697
1403,KODEX 구리선물(H),6280,-20,-0.32,50,15192
1404,JW중외제약우,30550,1550,5.34,50,22502
1405,KINDEX 국채선물3년인버스,99780,0,0.00,50,11


#### Dark Sky API
- 날씨 정보를 알려주는 API

In [17]:
import forecastio
FORECAST_TOKEN = '12b8cd89392777d5f1a9b56e91d172d5'

In [27]:
def forecast(lat, lng):    
    url = "https://api.darksky.net/forecast/{}/{},{}".format(FORECAST_TOKEN, lat, lng)   
    response = requests.get(url)    
    json_info = response.json()    
    return json_info["hourly"]["summary"], json_info["timezone"]

In [28]:
lat = 37.5124413 
lng = 126.9540519 
forecast(lat,lng)

('Partly cloudy until tomorrow morning.', 'Asia/Seoul')

In [29]:
def forecast(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHour = forecast.hourly()
    return byHour.summary

In [30]:
lat = 37.5124413 
lng = 126.9540519 
forecast(lat,lng)

'Partly cloudy until tomorrow morning.'

#### BeautifulSoup
- 네이버 검색어 순위
- bs4사용: html element를 selector를 사용한다.
- 네이버 검색어 순위를 가져와 데이터 프레임 만들기

In [31]:
def naver_top20():
    df = pd.DataFrame(columns = ['rank', 'keyword'])
    response = requests.get('http://naver.com')
    dom = BeautifulSoup(response.content, 'html.parser')
    keywords = dom.select('.ah_roll .ah_l .ah_item')
    for keyword in keywords:
        df.loc[len(df)] = {
            'rank': keyword.select_one('.ah_r').text,
            'keyword': keyword.select_one('.ah_k').text,
        }
    return df
    

In [32]:
naver_df = naver_top20()
naver_df

,rank,keyword
0,1,조재현
1,2,정승환
2,3,정병국
3,4,포르투갈 모로코
4,5,복지로
5,6,2001년 시트콤
6,7,재일교포 여배우
7,8,연애혁명
8,9,우루과이 사우디
9,10,이명희


#### 다음 검색어 순위

In [33]:
def daum_top10():
    df = pd.DataFrame(columns = ['rank', 'keyword'])
    response = requests.get('http://daum.net')
    dom = BeautifulSoup(response.content, 'html.parser')
    keywords = dom.select('#mArticle ol.list_hotissue.issue_row.list_mini > li')
    for keyword in keywords:
        df.loc[len(df)] = {
            'rank': keyword.select_one('.ir_wa').text.replace('위', ''),
            'keyword': keyword.select_one('.link_issue').text,
        }
    return df

In [34]:
daum_df = daum_top10()
daum_df

,rank,keyword
0,1,조재현
1,2,정병국
2,3,정승환
3,4,이명희
4,5,김갑수
5,6,전원책
6,7,김제동
7,8,유병재
8,9,지석진
9,10,재일교포 여배우


#### 중복된 키워드 찾아서 출력하기;

In [48]:
result = [keyword for keyword in naver_df['keyword'] if daum_df['keyword'].str.contains(keyword).any()]

In [49]:
result

['조재현', '정승환', '정병국', '재일교포 여배우', '이명희', '전원책', '김제동']

#### File DownLoad

In [ ]:
def download(title, download_link):
    response = requests.get(download_link, stream=True)
    download_path = './data/' + title
    size = 0
    with open(download_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                size +=1024
                f.write(chunk)
        